In [ ]:
# -*- coding: utf-8 -*-
"""Deploy Pendeteksi Burung dengan Flask di Google Colab"""

# Install library yang diperlukan
!pip install ultralytics flask pyngrok opencv-python-headless --quiet

# Import library
from flask import Flask, request, render_template_string, send_file
from pyngrok import ngrok
from ultralytics import YOLO
import cv2
import os
import numpy as np

# Inisialisasi aplikasi Flask
app = Flask(__name__)

# Buat folder untuk menyimpan file sementara
os.makedirs('static/uploads', exist_ok=True)
os.makedirs('static/outputs', exist_ok=True)

# Muat model YOLOv8
model = YOLO('yolov8n.pt')

# Template HTML untuk antarmuka web
HTML_TEMPLATE = """
<!DOCTYPE html>
<html>
<head>
    <title>Pendeteksi Burung</title>
    <style>
        body { font-family: Arial, sans-serif; text-align: center; padding: 20px; }
        h1 { color: #333; }
        .container { max-width: 800px; margin: auto; }
        .result { margin-top: 20px; }
        img, video { max-width: 100%; height: auto; }
        input[type=file] { margin: 20px 0; }
        button { padding: 10px 20px; background-color: #4CAF50; color: white; border: none; cursor: pointer; }
        button:hover { background-color: #45a049; }
        .error { color: red; }
    </style>
</head>
<body>
    <div class="container">
        <h1>Pendeteksi Burung dengan YOLOv8</h1>
        <p>Unggah gambar (.jpg, .png) atau video (.mp4) untuk mendeteksi burung.</p>
        <form method="post" enctype="multipart/form-data">
            <input type="file" name="file" accept="image/*,video/*" required>
            <br>
            <button type="submit">Unggah dan Deteksi</button>
        </form>
        {% if error %}
            <p class="error">{{ error }}</p>
        {% endif %}
        {% if output_path %}
            <div class="result">
                <h3>Hasil Deteksi</h3>
                {% if is_image %}
                    <img src="{{ output_path }}" alt="Hasil Deteksi">
                {% else %}
                    <video controls>
                        <source src="{{ output_path }}" type="video/mp4">
                        Maaf, browser Anda tidak mendukung video.
                    </video>
                {% endif %}
                <br>
                <a href="{{ output_path }}" download>Unduh Hasil</a>
            </div>
        {% endif %}
    </div>
</body>
</html>
"""

# Fungsi untuk memproses gambar
def process_image(image_path, output_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    results = model(image_path)

    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = box.conf[0].item()
            class_id = int(box.cls[0].item())
            label = model.names[class_id]

            if label == 'bird':
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img, f'{label} {confidence:.2f}', (x1, y1 - 10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    cv2.imwrite(output_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    return output_path

# Fungsi untuk memproses video
def process_video(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame)

        for r in results:
            for box in r.boxes:
                cls = int(box.cls[0])
                label = model.names[cls]
                if label == 'bird':
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    conf = float(box.conf[0])
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, f'{label} {conf:.2f}', (x1, y1 - 10),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        out.write(frame)

    cap.release()
    out.release()
    return output_path

# Route utama
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        if 'file' not in request.files:
            return render_template_string(HTML_TEMPLATE, error='Tidak ada file yang diunggah')

        file = request.files['file']
        if file.filename == '':
            return render_template_string(HTML_TEMPLATE, error='Pilih file terlebih dahulu')

        filename = file.filename
        if not filename.lower().endswith(('.jpg', '.jpeg', '.png', '.mp4')):
            return render_template_string(HTML_TEMPLATE, error='Format file tidak didukung. Gunakan .jpg, .png, atau .mp4')

        input_path = os.path.join('static/uploads', filename)
        file.save(input_path)

        output_filename = 'output_' + filename
        output_path = os.path.join('static/outputs', output_filename)

        is_image = filename.lower().endswith(('.jpg', '.jpeg', '.png'))
        try:
            if is_image:
                process_image(input_path, output_path)
            else:
                process_video(input_path, output_path)
        except Exception as e:
            return render_template_string(HTML_TEMPLATE, error=f'Error saat memproses file: {str(e)}')

        return render_template_string(HTML_TEMPLATE, output_path='/' + output_path, is_image=is_image)

    return render_template_string(HTML_TEMPLATE)

# Route untuk file statis
@app.route('/static/<path:filename>')
def serve_static(filename):
    return send_file(os.path.join('static', filename))

# Setup ngrok dengan token Anda
!ngrok authtoken 2zKEqZK0KddGK3qLuEVOv2hxG1C_3M8KfiNoowgXg5FvqTKUC  # Token Anda
public_url = ngrok.connect(5000).public_url
print(f"🌐 Aplikasi berjalan di: {public_url}")

# Jalankan aplikasi Flask
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 5

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 5

🌐 Aplikasi berjalan di: https://628a-35-236-189-183.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an a

...

ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.


0: 640x384 1 bird, 157.6ms
Speed: 2.3ms preprocess, 157.6ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bird, 153.3ms
Speed: 2.3ms preprocess, 153.3ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bird, 147.0ms
Speed: 2.2ms preprocess, 147.0ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bird, 142.6ms
Speed: 3.4ms preprocess, 142.6ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bird, 142.7ms
Speed: 2.5ms preprocess, 142.7ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bird, 143.2ms
Speed: 3.0ms preprocess, 143.2ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bird, 148.1ms
Speed: 2.4ms preprocess, 148.1ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 bird, 156.7ms
Speed: 2.5ms preprocess, 156.7ms inference, 1.9ms postprocess per image at shape (1, 3, 640

INFO:werkzeug:127.0.0.1 - - [02/Jul/2025 17:33:00] "POST / HTTP/1.1" 200 -
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/us


image 1/1 /content/static/uploads/burunnggg.jpg: 384x640 1 bird, 1 cat, 1 dining table, 221.3ms
Speed: 6.3ms preprocess, 221.3ms inference, 1.9ms postprocess per image at shape (1, 3, 384, 640)


INFO:werkzeug:127.0.0.1 - - [02/Jul/2025 17:36:28] "POST / HTTP/1.1" 200 -
ERROR:root:Unexpected exception finding object shape
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/google/colab/_debugpy_repr.py", line 54, in get_shape
    shape = getattr(obj, 'shape', None)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 318, in __get__
    obj = instance._get_current_object()
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/werkzeug/local.py", line 519, in _get_current_object
    raise RuntimeError(unbound_message) from None
RuntimeError: Working outside of request context.

This typically means that you attempted to use functionality that needed
an active HTTP request. Consult the documentation on testing for
information about how to avoid this problem.
INFO:werkzeug:127.0.0.1 - - [02/Jul/2025 17:36:29] "GET /static/outputs/output_burunnggg.jpg HTTP/1